## In this project, I will collect and visualize the video data of five major 4th generation kpop girl group YouTube channels.

# 1. Introduction
## 1.1 Background
Currently, the 4th generation kpop girl group is dominating the kpop scene in Korea, and globally. Gaining the momentum from Blackpink, now girl groups are more successful than boy groups in gaining mass popularity. I was curious how the most popular 4th gen girl groups (Newjeans, Aespa, Ive, Lesserafim, Nmixx) are creating contents for thier YouTube channels. Ever since BTS, YouTube channel contents became a critical part in creating a bond with fandom. It is one of the most important barometer for the fandom's power of the group.

I want to monitor the performance of each group's content by collecting the video metric data for each channel, and creating a Tableau dashboard. The scope of the project will be limited to the 5 major 4th gen girl group channels. 

## 1.2 Objectives
In this project, I will focus on learning the followings:

- Utilize private YouTube crawling package I developed to gather YouTube channel data
- Gather video data automatically using jupyter lab scheduler
- Store the data in google sheets using pygsheets libary
- Visualize the video data for these five channels via Tableau to answer these questions:
    - Which channel is performing the best at the moment?
    - What type of content gets the most views?
    - What type of content has the most engagement among the fans?
    - What are some contents that didn't perform well?

## 1.3 Project process
1. Get the channel video data, and comments data from girl group channels using private YouTube crawling package.
2. Preprocess data and engineer new features
3. Create a Tableau dashboard using google sheets as DB
4. Conclusions

## 1.4 Dataset
### Data Source
For this project, I obtained the dataset myself by utilizing private YouTube public crawler package.

### Data Limitation
The data is a real-world dataset, suitable for research purposes. However, some videos can have incorrect video length values because of the YouTube display ads that interferes with the crawler. 

Also, I did not collect comments, since the data is too large to store in google sheets, and takes a long time to collect with the crawler package that I am using.

### Import Libraries

In [21]:
import os
import sys
sys.path.insert(0, '/Users/minguyeo/Documents/coding/portfolio/')

from YTcrawling.package.YT_crawler_v1 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read & Save Google Sheet 
import pygsheets
from google.oauth2 import service_account
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# Window
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac
    rc('font', family='AppleGothic')
    
plt.rcParams['axes.unicode_minus'] = False   

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
google_sheets_creds = os.getenv('GOOGLE_SHEETS_CREDS')

# Use the environment variable
if google_sheets_creds:
    gc = pygsheets.authorize(service_account_file=google_sheets_creds)
else:
    print("Environment variable GOOGLE_SHEETS_CREDS is not set.")

## 1. Collect channel data using private crawler

### List of 5 major 4th gen girl groups channel id

In [22]:
### YouTube Channel Id of five major 4th gen girl groups (NEWJEANS, AESPA, IVE, LESSERAFIM, NMIXX)
cr_dic = {'UCMki_UkHb4qSc0qyEcOHHJw': 'NEWJEANS',
          'UC9GtSLeksfK4yuJ_g1lgQbg': 'AESPA',
          'UC-Fnix71vRP64WXeo0ikd0Q': 'IVE',
          'UCs-QBT4qkj_YiQw1ZntDO3g': 'LESSERAFIM',
          'UCnUAyD4t2LkvW68YrDh7fDg': 'NMIXX'}

channel_id = list(cr_dic.keys())

### Read in the google sheet for video data of the 5 channels

In [125]:
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1XPgKbblXTnwPJ5FNW4adxnGmrhMJllveFe4EfqWOc48/edit#gid=2084333239"
sid = spreadsheet_url.split("/d/")[1].split("/edit")[0]
sh = gc.open_by_key(sid)
wks_vid_metric = sh.worksheet_by_title('video_metric')
wks_vid_list = sh.worksheet_by_title('video_list')

In [126]:
# Get video_metric Sheet
vid_metric_df = wks_vid_metric.get_as_df(has_header=True , index_column=None , start='A1', end='AE1000000' , numerize=True , empty_value=np.nan)

print('(row, col): ', vid_metric_df.shape)
vid_metric_df.head(3)

(row, col):  (126, 12)


,video_id,cr,subscriber,channel_id,video_url,title,upload_date,view,comment,like,video_length,video_format
0,NB44sWLhAAQ,NEWJEANS,"6,230,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/NB44sWLhAAQ,"[About Jeans] 오늘을, 기록혠 2 ⊹ 시골에 간 혠스터의 오늘 | HYE...",2023. 12. 8,"340,295","1,500","25,294",1161,NaN
1,uB3al5c0gJ4,NEWJEANS,"6,230,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/uB3al5c0gJ4,[Light Jeans] 2023 Japan Music Show Behind | N...,2023. 12. 7,"230,081",766,"17,462",803,NaN
2,RdHR1cZZUR0,NEWJEANS,"6,230,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/RdHR1cZZUR0,[By Jeans] 'Sarah Kang - once in a moon' Cover...,2023. 12. 5,"882,501","5,200","89,583",409,NaN


In [94]:
# Get ad_check Sheet
vid_list_df = wks_vid_list.get_as_df(has_header=True , index_column=None , start='A1', end='AE1000000' , numerize=True , empty_value=np.nan)

print('(row, col): ', vid_list_df.shape)
vid_list_df.head(3)

(row, col):  (143, 4)


,channel_id,video_url,is_ad,video_format
0,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/NB44sWLhAAQ,F,VOD
1,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/uB3al5c0gJ4,F,VOD
2,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/RdHR1cZZUR0,F,VOD


### Load YouTube Crawler Class & get video stats

In [140]:
crawler = YoutubeCrawler()

In [27]:
# Store channels' video id lists (VOD)
vod_dic = crawler.get_videos(channel_id=channel_id, video_type="VOD", date_limit="10일 전")

  0%|          | 0/5 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCMki_UkHb4qSc0qyEcOHHJw/videos


 20%|██        | 1/5 [00:09<00:36,  9.06s/it]

4일 전 https://youtu.be/NB44sWLhAAQ
5일 전 https://youtu.be/uB3al5c0gJ4
7일 전 https://youtu.be/RdHR1cZZUR0
1 https://www.youtube.com/channel/UC9GtSLeksfK4yuJ_g1lgQbg/videos


 40%|████      | 2/5 [00:15<00:23,  7.70s/it]

...

1일 전 https://youtu.be/-MtfHxitugU
4일 전 https://youtu.be/IALcGjfZPF4
6일 전 https://youtu.be/Evz7xIaAKJQ
10일 전 https://youtu.be/l5LpxzgW4J0
3 https://www.youtube.com/channel/UCs-QBT4qkj_YiQw1ZntDO3g/videos


 80%|████████  | 4/5 [00:26<00:05,  5.95s/it]

1시간 전 https://youtu.be/MwCnjaAaA7Y
1일 전 https://youtu.be/GkdjGvgx4Bk
5일 전 https://youtu.be/Ml-n-jy9Ndc
6일 전 https://youtu.be/IfMUaBDO3UE
7일 전 https://youtu.be/ib5bV_dWs9A
4 https://www.youtube.com/channel/UCnUAyD4t2LkvW68YrDh7fDg/videos


100%|██████████| 5/5 [00:30<00:00,  6.17s/it]

1일 전 https://youtu.be/xu_XNASHbuU
4일 전 https://youtu.be/Ob92euLB_xc
6일 전 https://youtu.be/4_vzrcxpT4I
7일 전 https://youtu.be/_sNvoBDAaP0


In [28]:
# Store channels' video id lists (SHORTS)
shorts_dic = crawler.get_videos(channel_id=channel_id, video_type="Shorts", shorts_count=20)

  0%|          | 0/5 [00:00<?, ?it/s]

0 https://www.youtube.com/channel/UCMki_UkHb4qSc0qyEcOHHJw
0 https://www.youtube.com/channel/UCMki_UkHb4qSc0qyEcOHHJw/shorts


 20%|██        | 1/5 [00:12<00:48, 12.06s/it]

0 0 https://youtu.be/ygV508RkQEQ
0 1 https://youtu.be/JDlXOp5mGws
0 2 https://youtu.be/Mxv2P187m-k
0 3 https://youtu.be/MYcI76emM0k
0 4 https://youtu.be/Tjy4KsoTGTM
0 5 https://youtu.be/JhsdtEeT_VQ
0 6 https://youtu.be/kEmfaGm0Vn8
0 7 https://youtu.be/WThGkfg7wrI
0 8 https://youtu.be/WqmbxjkiTwQ
0 9 https://youtu.be/_JjR7tpYjBo
0 10 https://youtu.be/YOa6G9PCdEE
0 11 https://youtu.be/eeDLX0OdxpE
0 12 https://youtu.be/gMMscf4rdOQ
0 13 https://youtu.be/e9QZIohNubE
0 14 https://youtu.be/yRf55fFE4MQ
0 15 https://youtu.be/FyhdQl1ERoY
0 16 https://youtu.be/U6G6ZDKDvIQ
0 17 https://youtu.be/5m4UBXXs01s
0 18 https://youtu.be/xsYYxUsuXZY
0 19 https://youtu.be/-r0Vat3Paxw
1 https://www.youtube.com/channel/UC9GtSLeksfK4yuJ_g1lgQbg
1 https://www.youtube.com/channel/UC9GtSLeksfK4yuJ_g1lgQbg/shorts


 40%|████      | 2/5 [00:23<00:35, 11.77s/it]

...

2 0 https://youtu.be/Ou8szg318Lo
2 1 https://youtu.be/UIkmJDaDGQ0
2 2 https://youtu.be/gYzQVam9NJI
2 3 https://youtu.be/0s8_afbgbc0
2 4 https://youtu.be/HgmfWyG2wNg
2 5 https://youtu.be/YbH2SuTkpvo
2 6 https://youtu.be/hiaEA0wy1kI
2 7 https://youtu.be/VWiv2_QEvNA
2 8 https://youtu.be/3_rbbNR1yr0
2 9 https://youtu.be/WEVxf5qCQVA
2 10 https://youtu.be/K0zSAM2FQgc
2 11 https://youtu.be/HfB9MNdROHQ
2 12 https://youtu.be/mId1pnKVN6M
2 13 https://youtu.be/vRfBpJ0p0xI
2 14 https://youtu.be/woidWbTluWw
2 15 https://youtu.be/x8XbRTah5mw
2 16 https://youtu.be/2-hMMJGl8Cg
2 17 https://youtu.be/UmE5xh-__A4
2 18 https://youtu.be/6Nz10U5ROn8
2 19 https://youtu.be/r-YyRWdHgzo
3 https://www.youtube.com/channel/UCs-QBT4qkj_YiQw1ZntDO3g
3 https://www.youtube.com/channel/UCs-QBT4qkj_YiQw1ZntDO3g/shorts


 80%|████████  | 4/5 [00:44<00:10, 10.81s/it]

3 0 https://youtu.be/B-MFiegMJ3k
3 1 https://youtu.be/bAT8Ehz-osQ
3 2 https://youtu.be/Sn_IjuaFqIQ
3 3 https://youtu.be/c_RW4os40wk
3 4 https://youtu.be/kJ22Bq0oG08
3 5 https://youtu.be/Ie4DSJ8jAl0
3 6 https://youtu.be/oh7v2MiH0BU
3 7 https://youtu.be/sdp8zDEwiWc
3 8 https://youtu.be/HrQ2Q70zvKk
3 9 https://youtu.be/l-juemcpxP8
3 10 https://youtu.be/5R-SEvm7AUk
3 11 https://youtu.be/cI29jLDQhBw
3 12 https://youtu.be/uwGHVJyrG_U
3 13 https://youtu.be/JpmY8kAfij4
3 14 https://youtu.be/SkUJiAI5J9o
3 15 https://youtu.be/aUqx4aIrH-o
3 16 https://youtu.be/JWuKEM1iBHE
3 17 https://youtu.be/dRFLVtbrA9k
3 18 https://youtu.be/jjW9IZYBWrs
3 19 https://youtu.be/ust5_AvrRrM
4 https://www.youtube.com/channel/UCnUAyD4t2LkvW68YrDh7fDg
4 https://www.youtube.com/channel/UCnUAyD4t2LkvW68YrDh7fDg/shorts


100%|██████████| 5/5 [00:54<00:00, 10.86s/it]

4 0 https://youtu.be/_0YlvuwxlC0
4 1 https://youtu.be/nYe8ffhpE3Y
4 2 https://youtu.be/oV-w1MBedF0
4 3 https://youtu.be/ZybZEZSKVR8
4 4 https://youtu.be/3TgXHq7ZPqk
4 5 https://youtu.be/Kr7-MBdXYBc
4 6 https://youtu.be/PuvVuHyNVkU
4 7 https://youtu.be/jNqxOpIwrNk
4 8 https://youtu.be/3j_1U-g3z3g
4 9 https://youtu.be/O_fRI4veuq4
4 10 https://youtu.be/RFDMhqsSGhs
4 11 https://youtu.be/xSEpc_B4kNQ
4 12 https://youtu.be/zx6EA6wKVsk
4 13 https://youtu.be/IO_7mxzOn7Y
4 14 https://youtu.be/XHtD8zDOV8s
4 15 https://youtu.be/X0YiZVzfTws
4 16 https://youtu.be/zHdXWp__B18
4 17 https://youtu.be/9B4TBk7fUd4
4 18 https://youtu.be/55Jt0QTtQVo
4 19 https://youtu.be/Ejo8IsdRcK4


### Combine new video list with existing ad_list

In [88]:
vod_dic

{'UCMki_UkHb4qSc0qyEcOHHJw': ['https://youtu.be/NB44sWLhAAQ',
  'https://youtu.be/uB3al5c0gJ4',
  'https://youtu.be/RdHR1cZZUR0'],
 'UC9GtSLeksfK4yuJ_g1lgQbg': ['https://youtu.be/5rGgmvZIfus',
  'https://youtu.be/9Wmcywak2vQ',
  'https://youtu.be/2sVHcrZMYWo',
  'https://youtu.be/Yy3BCQCeBMg',
  'https://youtu.be/9J1lOMDYW14',
  'https://youtu.be/1ddlidZjBWg',
  'https://youtu.be/gvjzw8IABjU'],
 'UC-Fnix71vRP64WXeo0ikd0Q': ['https://youtu.be/-MtfHxitugU',
  'https://youtu.be/IALcGjfZPF4',
  'https://youtu.be/Evz7xIaAKJQ',
  'https://youtu.be/l5LpxzgW4J0'],
 'UCs-QBT4qkj_YiQw1ZntDO3g': ['https://youtu.be/MwCnjaAaA7Y',
  'https://youtu.be/GkdjGvgx4Bk',
  'https://youtu.be/Ml-n-jy9Ndc',
  'https://youtu.be/IfMUaBDO3UE',
  'https://youtu.be/ib5bV_dWs9A'],
 'UCnUAyD4t2LkvW68YrDh7fDg': ['https://youtu.be/xu_XNASHbuU',
  'https://youtu.be/Ob92euLB_xc',
  'https://youtu.be/4_vzrcxpT4I',
  'https://youtu.be/_sNvoBDAaP0']}

In [95]:
# Combine new vod list with existing list from google sheet(ad_list)
for i in range(len(vod_dic.keys())):
    # Create a df
    new_vod_df = pd.DataFrame({'channel_id':[list(vod_dic.keys())[i]] * len(vod_dic[list(vod_dic.keys())[i]]), 
                                  'video_url': vod_dic[list(vod_dic.keys())[i]],
                                  'video_format':'VOD'})
    vid_list_df = pd.concat([vid_list_df, new_vod_df])
    
vid_list_df = vid_list_df.drop_duplicates(subset=['video_url'], keep='first')
vid_list_df = vid_list_df.reset_index(drop=True)
vid_list_df

,channel_id,video_url,is_ad,video_format
0,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/NB44sWLhAAQ,F,VOD
1,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/uB3al5c0gJ4,F,VOD
2,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/RdHR1cZZUR0,F,VOD
3,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/Ol1K9K2HRqo,F,VOD
4,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/oMhrUjdkZRs,F,VOD
...,...,...,...,...
138,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/_0YlvuwxlC0,F,SHORTS
139,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/nYe8ffhpE3Y,F,SHORTS
140,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/oV-w1MBedF0,F,SHORTS
141,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/ZybZEZSKVR8,F,SHORTS


In [96]:
# Combine new shorts list with existing list from google sheet(ad_list)
for i in range(len(shorts_dic.keys())):
    # Create a df
    new_shorts_df = pd.DataFrame({'channel_id':[list(shorts_dic.keys())[i]] * len(shorts_dic[list(shorts_dic.keys())[i]]), 
                                  'video_url': shorts_dic[list(shorts_dic.keys())[i]],
                                  'video_format':'SHORTS'})
    vid_list_df = pd.concat([vid_list_df, new_shorts_df])
    
vid_list_df = vid_list_df.drop_duplicates(subset=['video_url'], keep='first')
vid_list_df = vid_list_df.reset_index(drop=True)
vid_list_df

,channel_id,video_url,is_ad,video_format
0,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/NB44sWLhAAQ,F,VOD
1,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/uB3al5c0gJ4,F,VOD
2,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/RdHR1cZZUR0,F,VOD
3,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/Ol1K9K2HRqo,F,VOD
4,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/oMhrUjdkZRs,F,VOD
...,...,...,...,...
138,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/_0YlvuwxlC0,F,SHORTS
139,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/nYe8ffhpE3Y,F,SHORTS
140,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/oV-w1MBedF0,F,SHORTS
141,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/ZybZEZSKVR8,F,SHORTS


### Update channel video list to google sheet

In [31]:
# vid_list sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1XPgKbblXTnwPJ5FNW4adxnGmrhMJllveFe4EfqWOc48/edit#gid=2084333239"
sid = spreadsheet_url.split("/d/")[1].split("/edit")[0]
sh = gc.open_by_key(sid)

# Update the sheet with new dataframe
wks_ad = sh.worksheet_by_title('video_list')
wks_ad.clear('A1','AZ')
wks_ad.set_dataframe(vid_list_df, 'A1', index=False)

### Load the video list again

In [100]:
# Get the dataframe back from the sheet
# vid_list sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1XPgKbblXTnwPJ5FNW4adxnGmrhMJllveFe4EfqWOc48/edit#gid=2084333239"
sid = spreadsheet_url.split("/d/")[1].split("/edit")[0]
sh = gc.open_by_key(sid)
wks_vid_list = sh.worksheet_by_title('video_list')

# Get ad_check Sheet
vid_list_df = wks_vid_list.get_as_df(has_header=True , index_column=None , start='A1', end='AE1000000' , numerize=True , empty_value=np.nan)

print('(row, col): ', vid_list_df.shape)
vid_list_df.head(3)

(row, col):  (143, 4)


,channel_id,video_url,is_ad,video_format
0,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/NB44sWLhAAQ,F,VOD
1,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/uB3al5c0gJ4,F,VOD
2,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/RdHR1cZZUR0,F,VOD


### Check if any of the new video is sponsored

In [103]:
# Get the new vid list from the total df
new_vid_list_df = vid_list_df.loc[vid_list_df['is_ad'].isnull()].reset_index(drop=True)

# Check for sponsored videos
new_vid_list_df = crawler.check_ad(new_vid_list_df)

  6%|▋         | 1/16 [00:12<03:01, 12.12s/it]

1 https://youtu.be/9Wmcywak2vQ False


 12%|█▎        | 2/16 [00:21<02:27, 10.52s/it]

2 https://youtu.be/-MtfHxitugU False


 19%|█▉        | 3/16 [00:30<02:08,  9.89s/it]

...

14 https://youtu.be/oV-w1MBedF0 False


 94%|█████████▍| 15/16 [02:16<00:09,  9.47s/it]

15 https://youtu.be/ZybZEZSKVR8 False


100%|██████████| 16/16 [02:24<00:00,  9.04s/it]

16 https://youtu.be/3TgXHq7ZPqk False


In [106]:
# Get the total video list
vid_list_df = pd.concat([vid_list_df, new_vid_list_df])
vid_list_df = vid_list_df.drop_duplicates(subset=['video_url'], keep='last')
vid_list_df = vid_list_df.reset_index(drop=True)

# vid_list sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1XPgKbblXTnwPJ5FNW4adxnGmrhMJllveFe4EfqWOc48/edit#gid=2084333239"
sid = spreadsheet_url.split("/d/")[1].split("/edit")[0]
sh = gc.open_by_key(sid)

# Update the sheet with new dataframe
wks_ad = sh.worksheet_by_title('video_list')
wks_ad.clear('A1','AZ')
wks_ad.set_dataframe(vid_list_df, 'A1', index=False)

### Get video metrics for the new video list

In [107]:
new_vid_metric_df = crawler.get_metrics(new_vid_list_df)

  6%|▌         | 1/17 [00:08<02:08,  8.01s/it]

0 UC9GtSLeksfK4yuJ_g1lgQbg 508만 https://youtu.be/5rGgmvZIfus 되게 꽉 차게 하루를 보냈던 것 같아요🐶🍀👚 | NINGNING in Atlanta & Washington & Chicago | NING’S LOG 2023.12.11 124,646 662 12,150


 12%|█▏        | 2/17 [00:15<01:57,  7.86s/it]

1 UC9GtSLeksfK4yuJ_g1lgQbg 508만 https://youtu.be/9Wmcywak2vQ 첫 개인화보를 오늘 찍었는데요😎🖤 | aespa GISELLE W Korea Photoshoot Behind 2023.12.10 58,614 549 8,582


 18%|█▊        | 3/17 [00:23<01:50,  7.91s/it]

...

14 UCnUAyD4t2LkvW68YrDh7fDg 228만 https://youtu.be/oV-w1MBedF0 sᴀɴᴛᴀ ᴛᴇʟʟ ᴍᴇ👧  ɪғ ʏᴏᴜ'ʀᴇ ʀᴇᴀʟʟʏ ᴛʜᴇʀᴇ ˗ˋ꒰🎅🏻꒱ˊ˗ #NMIXX #엔믹스 #설윤 #SULLYOON #규진 #KYUJIN #Soñar 2023.12.10 218,013 162 33,118


 94%|█████████▍| 16/17 [02:03<00:07,  7.61s/it]

15 UCnUAyD4t2LkvW68YrDh7fDg 228만 https://youtu.be/ZybZEZSKVR8 권민우 변호사, 로스쿨 시절 별명이 뭐였는지 알아? 🐨🐱 Come a so #Soñar 🌪#NMIXX #LILY #KYUJIN #Soñar #Soñar_Challenge 2023.12.10 293,849 252 27,148


100%|██████████| 17/17 [02:10<00:00,  7.71s/it]

16 UCnUAyD4t2LkvW68YrDh7fDg 228만 https://youtu.be/3TgXHq7ZPqk What a chill #Soñar🐿️ with #RedVelvet #레드벨벳 #WENDY #웬디#NMIXX #SULLYOON #Soñar_Challenge @redvelvet 2023.12.10 484,438 198 62,770


### Update video_length

In [121]:
# Add cr, video_length, video_format column to new_vid_metric_df
# Use the map method to create a new 'channel_name' column based on 'channel_id'
new_vid_metric_df['cr'] = new_vid_metric_df['channel_id'].map(cr_dic)
new_vid_metric_df['video_length']=np.nan

In [141]:
final_vid_metric_df = crawler.get_vid_length(new_vid_metric_df)

  0%|          | 0/17 [00:00<?, ?it/s]

0 https://www.youtube.com/results?search_query=https://youtu.be/5rGgmvZIfus AESPA https://youtu.be/5rGgmvZIfus


  6%|▌         | 1/17 [00:15<04:06, 15.40s/it]

14:58
1 https://www.youtube.com/results?search_query=https://youtu.be/9Wmcywak2vQ AESPA https://youtu.be/9Wmcywak2vQ


 12%|█▏        | 2/17 [00:29<03:35, 14.35s/it]

...

SHORTS
15 https://www.youtube.com/results?search_query=https://youtu.be/ZybZEZSKVR8 NMIXX https://youtu.be/ZybZEZSKVR8


 94%|█████████▍| 16/17 [03:29<00:12, 12.82s/it]

SHORTS
16 https://www.youtube.com/results?search_query=https://youtu.be/3TgXHq7ZPqk NMIXX https://youtu.be/3TgXHq7ZPqk


100%|██████████| 17/17 [03:42<00:00, 13.08s/it]

SHORTS


In [157]:
final_vid_metric_df

,video_id,cr,subscriber,channel_id,video_url,title,upload_date,view,comment,like,video_length
0,5rGgmvZIfus,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/5rGgmvZIfus,되게 꽉 차게 하루를 보냈던 것 같아요🐶🍀👚 | NINGNING in Atlanta...,2023-12-11,"124,646",662,"12,150",14:58
1,9Wmcywak2vQ,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/9Wmcywak2vQ,첫 개인화보를 오늘 찍었는데요😎🖤 | aespa GISELLE W Korea Pho...,2023-12-10,"58,614",549,"8,582",5:57
2,-MtfHxitugU,IVE,"3,170,000",UC-Fnix71vRP64WXeo0ikd0Q,https://youtu.be/-MtfHxitugU,[IVE ON] 'I'VE MINE' JAPAN PROMOTION BEHIND #1,2023-12-10,"96,082",342,"7,443",10:07
3,MwCnjaAaA7Y,LESSERAFIM,"4,270,000",UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/MwCnjaAaA7Y,[LE▶️PLAY] 다꾸라 초콜릿 VS 아기 밀크초콜릿🍫 | 꾸라은즈의 대결⁉️,2023-12-12,"36,095",395,"6,196",2:03
4,GkdjGvgx4Bk,LESSERAFIM,"4,270,000",UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/GkdjGvgx4Bk,[EPISODE] KIM CHAEWON ‘짐빔 하이볼’ Ads Shoot Sketch,2023-12-11,"112,963",529,"10,090",12:31
5,xu_XNASHbuU,NMIXX,"2,280,000",UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/xu_XNASHbuU,NMIXX(엔믹스) ‘Soñar (Breaker)’ Recording Behind ...,2023-12-11,"198,390",699,"19,493",14:47
6,ygV508RkQEQ,NEWJEANS,"6,240,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/ygV508RkQEQ,📦 I'm waiting for you #NewJeans #뉴진스 #SeasonsG...,2023-12-11,"88,648",699,"22,324",SHORTS
7,JDlXOp5mGws,NEWJEANS,"6,240,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/JDlXOp5mGws,📦 I'm waiting for you #NewJeans #뉴진스 #SeasonsG...,2023-12-11,"90,950",699,"23,132",SHORTS
8,72ARETXlWqI,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/72ARETXlWqI,크리스마스 선물로 #audiz 어떤데?#aespa #에스파 #KARINA #카리나 ...,2023-12-11,"384,967",427,"64,633",SHORTS
9,fRUmGjnmLxc,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/fRUmGjnmLxc,#첫눈 에 널 사랑하게 됐고 영원할 거라는 걸 알고 있었어🔮#첫눈챌린지 ❄️ #ae...,2023-12-11,"561,034",427,"59,896",SHORTS


In [160]:
# # merge with new_vid_list_df to get video_format values
final_vid_metric_df = pd.merge(final_vid_metric_df, new_vid_list_df[['video_url','video_format']], how='left', on='video_url')

# Reorder the df columns
final_vid_metric_df = final_vid_metric_df[['video_id', 'cr', 'subscriber','channel_id','video_url', 'title',
       'upload_date', 'view', 'comment', 'like', 'video_length', 'video_format']]

In [161]:
final_vid_metric_df

,video_id,cr,subscriber,channel_id,video_url,title,upload_date,view,comment,like,video_length,video_format
0,5rGgmvZIfus,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/5rGgmvZIfus,되게 꽉 차게 하루를 보냈던 것 같아요🐶🍀👚 | NINGNING in Atlanta...,2023-12-11,"124,646",662,"12,150",14:58,VOD
1,9Wmcywak2vQ,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/9Wmcywak2vQ,첫 개인화보를 오늘 찍었는데요😎🖤 | aespa GISELLE W Korea Pho...,2023-12-10,"58,614",549,"8,582",5:57,VOD
2,-MtfHxitugU,IVE,"3,170,000",UC-Fnix71vRP64WXeo0ikd0Q,https://youtu.be/-MtfHxitugU,[IVE ON] 'I'VE MINE' JAPAN PROMOTION BEHIND #1,2023-12-10,"96,082",342,"7,443",10:07,VOD
3,MwCnjaAaA7Y,LESSERAFIM,"4,270,000",UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/MwCnjaAaA7Y,[LE▶️PLAY] 다꾸라 초콜릿 VS 아기 밀크초콜릿🍫 | 꾸라은즈의 대결⁉️,2023-12-12,"36,095",395,"6,196",2:03,VOD
4,GkdjGvgx4Bk,LESSERAFIM,"4,270,000",UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/GkdjGvgx4Bk,[EPISODE] KIM CHAEWON ‘짐빔 하이볼’ Ads Shoot Sketch,2023-12-11,"112,963",529,"10,090",12:31,VOD
5,xu_XNASHbuU,NMIXX,"2,280,000",UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/xu_XNASHbuU,NMIXX(엔믹스) ‘Soñar (Breaker)’ Recording Behind ...,2023-12-11,"198,390",699,"19,493",14:47,VOD
6,ygV508RkQEQ,NEWJEANS,"6,240,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/ygV508RkQEQ,📦 I'm waiting for you #NewJeans #뉴진스 #SeasonsG...,2023-12-11,"88,648",699,"22,324",SHORTS,VOD
7,JDlXOp5mGws,NEWJEANS,"6,240,000",UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/JDlXOp5mGws,📦 I'm waiting for you #NewJeans #뉴진스 #SeasonsG...,2023-12-11,"90,950",699,"23,132",SHORTS,VOD
8,72ARETXlWqI,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/72ARETXlWqI,크리스마스 선물로 #audiz 어떤데?#aespa #에스파 #KARINA #카리나 ...,2023-12-11,"384,967",427,"64,633",SHORTS,SHORTS
9,fRUmGjnmLxc,AESPA,"5,080,000",UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/fRUmGjnmLxc,#첫눈 에 널 사랑하게 됐고 영원할 거라는 걸 알고 있었어🔮#첫눈챌린지 ❄️ #ae...,2023-12-11,"561,034",427,"59,896",SHORTS,SHORTS


### Close driver

In [162]:
crawler.close_driver()

### Combine new video data with existing data

In [163]:
final_df = pd.concat([vid_metric_df, final_vid_metric_df])
final_df = final_df.drop_duplicates(subset=['video_id'], keep='first')
final_df = final_df.reset_index(drop=True)

### Change data types

In [164]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      143 non-null    object
 1   cr            143 non-null    object
 2   subscriber    143 non-null    object
 3   channel_id    143 non-null    object
 4   video_url     143 non-null    object
 5   title         143 non-null    object
 6   upload_date   143 non-null    object
 7   view          143 non-null    object
 8   comment       143 non-null    object
 9   like          143 non-null    object
 10  video_length  142 non-null    object
 11  video_format  17 non-null     object
dtypes: object(12)
memory usage: 13.5+ KB


In [165]:
# Change subscriber and view metric to int type
to_number = ['subscriber','view','comment','like']
for col in to_number:
    final_df[col] = final_df[col].apply(lambda x: pd.to_numeric(x.replace(',',"")) if ',' in str(x) else pd.to_numeric(x))

In [166]:
# Change upload_date to datetime and video_length to seconds
final_df['upload_date'] = pd.to_datetime(final_df['upload_date'])

In [169]:
final_df.tail(20)

,video_id,cr,subscriber,channel_id,video_url,title,upload_date,view,comment,like,video_length,video_format
123,ERNNoKgjNac,NMIXX,2270000,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/ERNNoKgjNac,Come a so-soñar🌪️ #NMIXX #엔믹스 #Fe3O4_BREAK #So...,2023-12-03,1236532,1200,101031,60,NaN
124,eGkWGrRYw10,NMIXX,2270000,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/eGkWGrRYw10,배이&지우의 나는 환승 솔로 하트 연애 시그널💗#NMIXXIN무한도전 #Shorts,2023-09-25,53990,43,7287,60,NaN
125,FcIu-ECxnsk,NMIXX,2270000,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/FcIu-ECxnsk,"Oh My loooveee, My daaaarrling✨#NMIXXIN무한도전 #S...",2023-09-25,37999,37,5998,2365,NaN
126,5rGgmvZIfus,AESPA,5080000,UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/5rGgmvZIfus,되게 꽉 차게 하루를 보냈던 것 같아요🐶🍀👚 | NINGNING in Atlanta...,2023-12-11,124646,662,12150,14:58,VOD
127,9Wmcywak2vQ,AESPA,5080000,UC9GtSLeksfK4yuJ_g1lgQbg,https://youtu.be/9Wmcywak2vQ,첫 개인화보를 오늘 찍었는데요😎🖤 | aespa GISELLE W Korea Pho...,2023-12-10,58614,549,8582,5:57,VOD
128,-MtfHxitugU,IVE,3170000,UC-Fnix71vRP64WXeo0ikd0Q,https://youtu.be/-MtfHxitugU,[IVE ON] 'I'VE MINE' JAPAN PROMOTION BEHIND #1,2023-12-10,96082,342,7443,10:07,VOD
129,MwCnjaAaA7Y,LESSERAFIM,4270000,UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/MwCnjaAaA7Y,[LE▶️PLAY] 다꾸라 초콜릿 VS 아기 밀크초콜릿🍫 | 꾸라은즈의 대결⁉️,2023-12-12,36095,395,6196,2:03,VOD
130,GkdjGvgx4Bk,LESSERAFIM,4270000,UCs-QBT4qkj_YiQw1ZntDO3g,https://youtu.be/GkdjGvgx4Bk,[EPISODE] KIM CHAEWON ‘짐빔 하이볼’ Ads Shoot Sketch,2023-12-11,112963,529,10090,12:31,VOD
131,xu_XNASHbuU,NMIXX,2280000,UCnUAyD4t2LkvW68YrDh7fDg,https://youtu.be/xu_XNASHbuU,NMIXX(엔믹스) ‘Soñar (Breaker)’ Recording Behind ...,2023-12-11,198390,699,19493,14:47,VOD
132,ygV508RkQEQ,NEWJEANS,6240000,UCMki_UkHb4qSc0qyEcOHHJw,https://youtu.be/ygV508RkQEQ,📦 I'm waiting for you #NewJeans #뉴진스 #SeasonsG...,2023-12-11,88648,699,22324,SHORTS,VOD


In [171]:
# Function to convert video_length to seconds
def convert_to_seconds(length):
    
    if isinstance(length, (int,float)):
        return length # If already integer, then just return length
    if length == 'SHORTS':
        return 60  # Assuming SHORTS videos is approx 60 seconds
    else:
        parts = length.split(':')
        if len(parts) == 3:
            hours, minutes, seconds = map(int, parts)
        elif len(parts) == 2:
            hours, minutes, seconds = 0, int(parts[0]), int(parts[1])
        elif len(parts) == 1:
            hours, minutes, seconds = 0, 0, int(parts[0])
        else:
            return 0  # Invalid format
        return hours * 3600 + minutes * 60 + seconds

# Apply the conversion function to the DataFrame
final_df['video_length'] = final_df['video_length'].apply(convert_to_seconds)

### Update new video metrics to video_metric Sheet

In [172]:
# vid_list sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1XPgKbblXTnwPJ5FNW4adxnGmrhMJllveFe4EfqWOc48/edit#gid=2084333239"
sid = spreadsheet_url.split("/d/")[1].split("/edit")[0]
sh = gc.open_by_key(sid)

# Update the sheet with new dataframe
wks_ad = sh.worksheet_by_title('video_metric')
wks_ad.clear('A1','AZ')
wks_ad.set_dataframe(final_df, 'A1', index=False)